In [1]:
import sqlite3
import pprint as pp
import numpy as np
from sklearn.neighbors import NearestNeighbors
import datetime

In [2]:
def readTags(path, query):  
    conn = sqlite3.connect(path)
    res = conn.execute(query)
    tags = res.fetchall()
    # pp.pprint(data)
    print "Total: ", len(tags)
    return conn, tags

In [3]:
def createTermsIndices(tags):
    tagsDict = {}
    for index in range(len(tags)):
        tagsDict[tags[index][0]] = index
    print len(tagsDict.values())
    return tagsDict

In [4]:
def getArtistTerms(artist):
    query="SELECT term FROM artist_term where artist_id='%s';" %(artist)
    res = term.execute(query)
    artistTerms = res.fetchall()
    return artistTerms

In [5]:
def Timer():
    return datetime.datetime.now()

In [6]:
def createMatrix(artists, terms):
    mat = np.zeros((len(artists),len(terms)))
    print mat.shape

    for artistIndex in range(len(artists)):
        if artistIndex%10000 == 0:
            print "Processed %d items" %(artistIndex)
        artistTerms = getArtistTerms(artists[artistIndex])
        for artistTermIndex in range(len(artistTerms)):
            if artistTerms[artistTermIndex][0] in termsDict.keys():
    #             print termsDict[artistTerms[artistTermIndex][0]], artistTerms[artistTermIndex][0]
                mat[artistIndex][(termsDict[artistTerms[artistTermIndex][0]])]=1.0
    return mat

In [7]:
def createTable(filename, tablename):
    createConn = sqlite3.connect(filename)
    createTableQuery = """ CREATE TABLE IF NOT EXISTS %s (
                                        target text,
                                        similar text
                                    ); """ %(tablename)

    createConn.execute(createTableQuery)
    return createConn

In [8]:
def insertTable(createConn, tablename, indices, knn):
    counter = 0
    for currentIndex in indices:
        counter = counter+1
        if counter%10000 == 0:
            print "Inserted %d items" %(counter)
        for index in range(1, len(currentIndex)):
            if index == knn:
                break
    #         print artists[currentIndex[0]][0], artists[currentIndex[index]][0]
            insertQuery = """ INSERT INTO %s VALUES ('%s', '%s'); """ %(tablename, artists[currentIndex[0]][0], artists[currentIndex[index]][0])
    #         print insertQuery
            createConn.execute(insertQuery)
            

In [9]:
def checkTable(createConn, tablename):
    res =  createConn.execute("Select count(*) from %s;" %(tablename))
    print res.fetchall()

def deleteTable(createConn):
    createConn.execute("drop table %s;" %(tablename))

In [10]:
term, artists = readTags('data/artist_term.db', 'SELECT artist_id FROM artists;')
term, terms = readTags('data/artist_term.db',
                        'SELECT term FROM artist_term GROUP BY term HAVING count(*)>500;')
termsDict = createTermsIndices(terms)
mat = createMatrix(artists, terms)

Total:  44745
Total:  353
353
(44745, 353)
Processed 0 items
Processed 10000 items
Processed 20000 items
Processed 30000 items
Processed 40000 items


In [11]:
start = Timer()
print start
nbrs = NearestNeighbors(n_neighbors=200, algorithm='kd_tree').fit(mat)
distances, indices = nbrs.kneighbors(mat)
print indices
print distances
end = Timer()
print end - start

2017-11-27 01:45:26.206500
[[    0 19676 34301 ..., 36414 44146  6335]
 [    1 11189   294 ...,  4867  7635 33208]
 [    2 32635  6864 ..., 31604 33477  1280]
 ..., 
 [44742 33447 43158 ..., 12288 12319 12272]
 [10209 14681 13693 ..., 13696 13714 11131]
 [44744 36796 41754 ..., 11666   366 11607]]
[[ 0.          3.87298335  3.87298335 ...,  4.58257569  4.58257569
   4.69041576]
 [ 0.          2.          2.         ...,  2.64575131  2.64575131
   2.64575131]
 [ 0.          5.          5.38516481 ...,  6.32455532  6.32455532
   6.32455532]
 ..., 
 [ 0.          2.82842712  2.82842712 ...,  3.46410162  3.46410162
   3.46410162]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          3.          3.         ...,  3.74165739  3.74165739
   3.74165739]]
0:23:15.067246


## Calculating Accuracy

In [27]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'kdtree_tags_60')
checkTable(createConn, 'kdtree_tags_60')
insertTable(createConn, 'kdtree_tags_60', indices, 60)
checkTable(createConn, 'kdtree_tags_60')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(2639955,)]
0:00:51.528511


In [28]:
start = Timer()
learning_factor=7.3
total = 0.00
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM kdtree_tags_60 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:00.717704
0:00:01.492397
0:00:02.206296
0:00:02.879320
0:00:03.572623
0:00:04.259198
0:00:04.927166
0:00:05.595871
0:00:06.276206
0:00:06.938854
0:00:07.613484
0:00:08.272807
0:00:08.928032
0:00:09.594604
0:00:10.279914
0:00:10.930463
0:00:11.598102
0:00:12.250952
0:00:12.918110
0:00:13.576493


In [29]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target LIMIT 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)
print denominator

141.0


In [30]:
print "Accuracy Score for KD-Tree for 60 Neighbours", total/denominator
createConn.close()

Accuracy Score for KD-Tree for 60 Neighbours 0.695035460993


In [31]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'kdtree_tags_45')
checkTable(createConn, 'kdtree_tags_45')
insertTable(createConn, 'kdtree_tags_45', indices, 45)
checkTable(createConn, 'kdtree_tags_45')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(1968780,)]
0:00:37.523275


In [32]:
start = Timer()
total = 0.00
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM kdtree_tags_45 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:00.498710
0:00:00.995961
0:00:01.499149
0:00:02.074260
0:00:02.593661
0:00:03.107916
0:00:03.615770
0:00:04.097983
0:00:04.597751
0:00:05.086205
0:00:05.573392
0:00:06.070000
0:00:06.565796
0:00:07.069245
0:00:07.570539
0:00:08.082077
0:00:08.642745
0:00:09.192045
0:00:09.708714
0:00:10.233097


In [35]:
print "Accuracy Score for KD-Tree for 45 Neighbours", total/denominator 
createConn.close()

Accuracy Score for KD-Tree for 45 Neighbours 0.624113475177


In [37]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'kdtree_tags_90')
checkTable(createConn, 'kdtree_tags_90')
insertTable(createConn, 'kdtree_tags_90', indices, 90)
checkTable(createConn, 'kdtree_tags_90')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(3982305,)]
0:01:14.150480


In [38]:
start = Timer()
total = 0.00
learning_factor=5.2
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM kdtree_tags_90 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:01.087374
0:00:02.178647
0:00:03.220929
0:00:04.331187
0:00:05.390707
0:00:06.418387
0:00:07.477845
0:00:08.544607
0:00:09.638925
0:00:10.712642
0:00:11.831967
0:00:12.908494
0:00:13.946259
0:00:14.977219
0:00:16.011019
0:00:17.048861
0:00:18.065303
0:00:19.084537
0:00:20.112773
0:00:21.124378


In [39]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target LIMIT 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)
# print denominator

In [40]:
print "Accuracy Score for KD-Tree for 90 Neighbours", total/denominator #90 neighbours algorithm: Ball_tree
createConn.close()

Accuracy Score for KD-Tree for 90 Neighbours 0.577114427861
